In [22]:
import pandas as pd
df = pd.read_csv('keywords.csv', encoding = 'big5')

In [66]:
target = dict()
ma_dict = dict()
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)
market_date_list = []

ma_last = 0
for a_row in reader_market:
    date = datetime.strptime( a_row['date'], "%Y-%m-%d").date()
    two_day_returns = float(a_row[u'2日報酬'])
    price = float(a_row['price'])
    ma = float(a_row['5MA'])
    ma_trend = 1 if ma >=ma_last else -1
    market_date_list.append(date)
    ma_dict[date] = [(price-ma)/ma]
    ma_last = ma
    if two_day_returns <= 0 : 
        target[date] = 0
    elif two_day_returns > 0 :
        target[date] = 1


In [2]:
label_dict = {2:'up', 1:'flat', 0:'down'}

In [13]:
import csv 
from datetime import datetime
from datetime import timedelta

bbs_content_plus_title = dict()
bbs_number_of_document = dict()
'''
bbs_content_plus_title[date] = '文章字串'
'''

time_delta = timedelta(hours = -13.5)
item = 'bbs'

with open( r'.\data' + '\\' + item + '_all' + '.csv' , 'r', encoding = 'utf-8') as f:
    #use market date  to create a dictionary
    for date in market_date_list:
        bbs_content_plus_title[date] = {'up':'', 'flat':'', 'down':''}
        bbs_number_of_document[date] = {'up':0, 'flat':0, 'down':0}


    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
        adj_time = time+time_delta
        adj_date = (adj_time + timedelta(days = 2)).date()


        if  adj_date in market_date_list: 
            title = a_row['title']
            content = a_row['content']
            label = label_dict[int(float(a_row['label']))]
            title_plus_content = title + '＜' + content

            
            bbs_content_plus_title[adj_date][label] +=  '＜' + title_plus_content

            bbs_number_of_document[adj_date][label]  += 1


In [14]:
import csv 
from datetime import datetime
from datetime import timedelta

news_content_plus_title = dict()
news_number_of_document = dict()
'''
news_content_plus_title[date] = '文章字串'
'''

time_delta = timedelta(hours = -13.5)
item = 'news'

with open( r'.\data' + '\\' + item + '_all' + '.csv' , 'r', encoding = 'big5') as f:
    #use market date  to create a dictionary
    for date in market_date_list:
        news_content_plus_title[date] = {'up':'', 'flat':'', 'down':''}
        news_number_of_document[date] = {'up':0, 'flat':0, 'down':0}


    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M") 
        adj_time = time+time_delta
        adj_date = (adj_time + timedelta(days = 2)).date()


        if  adj_date in market_date_list: 
            title = a_row['title']
            content = a_row['content']
            label = label_dict[int(float(a_row['label']))]
            title_plus_content = title + '＜' + content

            
            news_content_plus_title[adj_date][label] +=  '＜' + title_plus_content

            news_number_of_document[adj_date][label]  += 1


In [15]:
import csv 
from datetime import datetime
from datetime import timedelta

forum_content_plus_title = dict()
forum_number_of_document = dict()
'''
forum_content_plus_title[date] = '文章字串'
'''

time_delta = timedelta(hours = -13.5)
item = 'forum'

with open( r'.\data' + '\\' + item + '_all' + '.csv' , 'r', encoding = 'utf-8') as f:
    #use market date  to create a dictionary
    for date in market_date_list:
        forum_content_plus_title[date] = {'up':'', 'flat':'', 'down':''}
        forum_number_of_document[date] = {'up':0, 'flat':0, 'down':0}


    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
        adj_time = time+time_delta
        adj_date = (adj_time + timedelta(days = 2)).date()


        if  adj_date in market_date_list: 
            
            title = a_row['title']
            content = a_row['content']
            label = label_dict[int(float(a_row['label']))]
            title_plus_content = title + '＜' + content

            
            forum_content_plus_title[adj_date][label] +=  '＜' + title_plus_content

            forum_number_of_document[adj_date][label]  += 1


In [67]:
new_features_dict = {}
for date in market_date_list:
    try:
        new_features_dict[date] =ma_dict[date + timedelta(days =-2)] 
    except:
        new_features_dict[date] = [0]
    
    
for item in ['news', 'bbs', 'forum']:    
    if item == 'bbs':
        article_number = bbs_number_of_document
    elif item == 'forum':
        article_number = forum_number_of_document
    else:
        article_number = news_number_of_document
        
    for date in market_date_list:
        
        try:
            new_features_dict[date].append((article_number[date]['up']-article_number[date]['down'])/(article_number[date]['up']+article_number[date]['flat']+article_number[date]['down']))
        except:
            new_features_dict[date].append(0)
            
            


# feature to df

In [389]:
features_dict = {}
for date in market_date_list:
    features_dict[date] = [] 
    
    
feature_list = []
for item in ['news', 'bbs', 'forum']:    
    if item == 'bbs':
        article = bbs_content_plus_title
        article_number = bbs_number_of_document
    elif item == 'forum':
        article = forum_content_plus_title
        article_number = forum_number_of_document
    else:
        article = news_content_plus_title
        article_number = news_number_of_document
        
    for a_class in ['up','flat', 'down']:
        keyword_list = df[item+ '_' + a_class].dropna().tolist()
        
        
        feature_list += keyword_list
                
        for date in market_date_list:
            for keyword in keyword_list:
                
                try:
                    feature_i = article[date][a_class].count(keyword) / article_number[date][a_class]
                except:
                    feature_i = 0
                features_dict[date].append(    feature_i )
                    
                    
df_ready = pd.DataFrame.from_dict(features_dict, orient = 'index', columns  = feature_list)
dropped_columns_index = []
for i in range(len(df_ready.columns)):
    if df_ready.iloc[:,i].tolist().count(0) == 732:
        dropped_columns_index.append(i)
kept_colums = list(range(len(df_ready.columns)))
for i in dropped_columns_index:
    kept_colums.remove(i)
df_ready3 = df_ready.iloc[:,kept_colums]
y = pd.DataFrame.from_dict(target, orient = 'index', columns  = ['target'])

In [289]:
new_features_dict

{datetime.date(2016, 1, 11): [0, 0.15625, -0.030303030303030304, 0.0],
 datetime.date(2016, 1, 12): [0,
  0.23684210526315788,
  0.0,
  -0.058823529411764705],
 datetime.date(2016, 1, 13): [-0.016612394712619664,
  0.358974358974359,
  -0.08695652173913043,
  0.0],
 datetime.date(2016, 1, 14): [-0.011478804650870483,
  0.25806451612903225,
  -0.23076923076923078,
  0.0],
 datetime.date(2016, 1, 15): [-0.00011398629762036159,
  0.33707865168539325,
  -0.045454545454545456,
  -0.05],
 datetime.date(2016, 1, 18): [0,
  -0.05263157894736842,
  -0.25925925925925924,
  -0.041666666666666664],
 datetime.date(2016, 1, 19): [0,
  0.2988505747126437,
  -0.34782608695652173,
  0.0],
 datetime.date(2016, 1, 20): [0.0037721223784753324,
  0.22764227642276422,
  -0.15,
  0.0],
 datetime.date(2016, 1, 21): [0.0071505576532303865,
  0.21481481481481482,
  -0.022727272727272728,
  0.09523809523809523],
 datetime.date(2016, 1, 22): [-0.00963389054869212,
  0.1297709923664122,
  -0.046875,
  -0.033333333

In [390]:
new_features = pd.DataFrame.from_dict(new_features_dict, orient = 'index', columns  = ['ma\%', 'news\%','bbs\%','forum\%'])
for i in [ 'ma\%', 'news\%','bbs\%','forum\%']:
    df_ready3[i] = new_features[i]

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [391]:
df_ready3['target'] = y

from sklearn.model_selection import train_test_split as tt_split

X = df_ready3.iloc[2:,0:-1]
y = df_ready3.iloc[2:,:]['target']
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=0)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
df_ready3.to_csv('two_date_data.csv', encoding= 'big5')

## RF {'n_estimators': 143, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_depth': 90, 'criterion': 'entropy', 'bootstrap': False}
## logistic {'penalty': 'l2', 'C': 0.9099099099099099}
## svm {'kernel': 'linear', 'degree': 2, 'C': 0.7187187187187187}

In [80]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score 

In [301]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 143,  min_samples_split = 4,  min_samples_leaf = 7,  max_depth = 90,  criterion =  'entropy' ,  bootstrap = True, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099)
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]

In [302]:
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for clf in clf_list:
    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)
    print(classification_report(y_true, y_pred, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6552    0.2794    0.3918        68
     class 1     0.5812    0.8718    0.6974        78

   micro avg     0.5959    0.5959    0.5959       146
   macro avg     0.6182    0.5756    0.5446       146
weighted avg     0.6157    0.5959    0.5551       146

              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



In [303]:
new_pred_list = []

for j in range(len(predictlist[0])):
    tem_list = []
    for i in range(3):
        tem_list.append(predictlist[i][j])
    if tem_list.count(0) > tem_list.count(1):
        new_pred_list.append(0)
    else:
        new_pred_list.append(1)

In [304]:
print(classification_report(y_true, new_pred_list, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.5690    0.4853    0.5238        68
     class 1     0.6023    0.6795    0.6386        78

   micro avg     0.5890    0.5890    0.5890       146
   macro avg     0.5856    0.5824    0.5812       146
weighted avg     0.5868    0.5890    0.5851       146



In [305]:
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))

In [306]:
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for i in range(len(clf_list)):
    clf = clf_list[i]
    if i == 4:
        continue
    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)
    print(classification_report(y_true, y_pred, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6552    0.2794    0.3918        68
     class 1     0.5812    0.8718    0.6974        78

   micro avg     0.5959    0.5959    0.5959       146
   macro avg     0.6182    0.5756    0.5446       146
weighted avg     0.6157    0.5959    0.5551       146

              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146

              precision    recall  f1-score   support

     class 0     0.4875    0.5735    0.5270        68
     class 1     0.5606    0.4744    0.5139        78

   micro avg     0.5205    0.5205    0.5205       146
   macro avg     0.5241    0.5239    0.5205       146
weighted avg     0.5266    0.5205    0.5200       146



In [318]:
go_true = []
go_predict = []

for j in range(len(predictlist[0])):
    tem_list = []
    
    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])
    
    if tem_list.count(0) >=4 :
        go_true.append(y_true[j])
        go_predict.append(0)
    elif tem_list.count(1) >=4:
        go_true.append(y_true[j])
        go_predict.append(1)
print(classification_report(go_true, go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6667    0.3333    0.4444        18
     class 1     0.6667    0.8889    0.7619        27

   micro avg     0.6667    0.6667    0.6667        45
   macro avg     0.6667    0.6111    0.6032        45
weighted avg     0.6667    0.6667    0.6349        45



In [319]:
#出手率
len(go_predict) / len(y_true)

0.3082191780821918

# without ma

In [444]:
y_no_ma = df_ready3.iloc[2:,:]['target']
X_no_ma = df_ready3.iloc[2:,0:-1].drop(columns=['ma\%'])
x_train,x_test,y_train,y_test = tt_split(X_no_ma,y_no_ma,test_size=0.2,random_state=0)

In [445]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 143,  min_samples_split = 4,  min_samples_leaf = 7,  max_depth = 90,  criterion =  'entropy' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099)
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]

In [323]:
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for clf in clf_list:
    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)
    print(classification_report(y_true, y_pred, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6071    0.2500    0.3542        68
     class 1     0.5678    0.8590    0.6837        78

   micro avg     0.5753    0.5753    0.5753       146
   macro avg     0.5875    0.5545    0.5189       146
weighted avg     0.5861    0.5753    0.5302       146

              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



In [324]:
new_pred_list = []

for j in range(len(predictlist[0])):
    tem_list = []
    for i in range(3):
        tem_list.append(predictlist[i][j])
    if tem_list.count(0) > tem_list.count(1):
        new_pred_list.append(0)
    else:
        new_pred_list.append(1)
print(classification_report(y_true, new_pred_list, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.5455    0.4412    0.4878        68
     class 1     0.5824    0.6795    0.6272        78

   micro avg     0.5685    0.5685    0.5685       146
   macro avg     0.5639    0.5603    0.5575       146
weighted avg     0.5652    0.5685    0.5623       146



In [325]:
go_true = []
go_predict = []
for j in range(len(predictlist[0])):
    tem_list = []
    for i in range(3):
        tem_list.append(predictlist[i][j])
    
    if tem_list.count(0) ==3:
        go_true.append(y_true[j])
        go_predict.append(0)
    elif tem_list.count(1) ==3:
        go_true.append(y_true[j])
        go_predict.append(1)
print(classification_report(go_true, go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6667    0.2759    0.3902        29
     class 1     0.6667    0.9130    0.7706        46

   micro avg     0.6667    0.6667    0.6667        75
   macro avg     0.6667    0.5945    0.5804        75
weighted avg     0.6667    0.6667    0.6236        75



In [326]:
#出手率
len(go_predict) / len(y_true)

0.5136986301369864

In [175]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
import numpy as np

n_neighbors  = [int(x) for x in np.linspace(start = 3, stop = 9, num = 4)]


p = [1,2,3] 

algorithm = [ 'ball_tree', 'kd_tree','brute']

leaf_size  = [int(x) for x in np.linspace(start = 2, stop = 60, num = 30)]


# Create the random grid
random_grid = {'n_neighbors': n_neighbors,
               'p' : p,
               'algorithm': algorithm,
               'leaf_size': leaf_size
              }

clf = KNeighborsClassifier()
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 200, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
clf_random.fit(x_train, y_train)
clf_random.best_params_

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   30.3s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed: 10.3min finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'p': 2, 'n_neighbors': 3, 'leaf_size': 48, 'algorithm': 'ball_tree'}

In [176]:
print(clf_random.best_score_ )
clf =  clf_random.best_estimator_ 

y_pred = clf.predict(x_test)
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

0.513182427063241
              precision    recall  f1-score   support

     class 0       0.49      0.57      0.53        68
     class 1       0.56      0.47      0.51        78

   micro avg       0.52      0.52      0.52       146
   macro avg       0.52      0.52      0.52       146
weighted avg       0.53      0.52      0.52       146



In [448]:
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))

In [457]:
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for i in range(len(clf_list)):
    clf = clf_list[i]

    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)
    print(classification_report(y_true, y_pred, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.6071    0.2500    0.3542        68
     class 1     0.5678    0.8590    0.6837        78

   micro avg     0.5753    0.5753    0.5753       146
   macro avg     0.5875    0.5545    0.5189       146
weighted avg     0.5861    0.5753    0.5302       146

              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146



c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     class 0     0.5574    0.5000    0.5271        68
     class 1     0.6000    0.6538    0.6258        78

   micro avg     0.5822    0.5822    0.5822       146
   macro avg     0.5787    0.5769    0.5764       146
weighted avg     0.5801    0.5822    0.5798       146

              precision    recall  f1-score   support

     class 0     0.4875    0.5735    0.5270        68
     class 1     0.5606    0.4744    0.5139        78

   micro avg     0.5205    0.5205    0.5205       146
   macro avg     0.5241    0.5239    0.5205       146
weighted avg     0.5266    0.5205    0.5200       146



## 4個分類機器一致通過

In [458]:
go_true = []
go_predict = []
target_names = ['class 0', 'class 1']

for j in range(len(predictlist[0])):
    tem_list = []
    
    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])
    
    if tem_list.count(0) ==4 :
        go_true.append(y_true[j])
        go_predict.append(0)
    elif tem_list.count(1) >=4:
        go_true.append(y_true[j])
        go_predict.append(1)
print(classification_report(go_true, go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.7778    0.4118    0.5385        17
     class 1     0.7059    0.9231    0.8000        26

   micro avg     0.7209    0.7209    0.7209        43
   macro avg     0.7418    0.6674    0.6692        43
weighted avg     0.7343    0.7209    0.6966        43



In [459]:
#出手率
len(go_predict) / len(y_true)

0.2945205479452055

# 看看不出手時的準確率

## 3對1

In [331]:
not_go_true = []
not_go_predict = []


for j in range(len(predictlist[0])):
    tem_list = []
    
    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])
    
    if tem_list.count(0) ==3 :
        not_go_true.append(y_true[j])
        not_go_predict.append(0)
    elif tem_list.count(1) ==3:
        not_go_true.append(y_true[j])
        not_go_predict.append(1)
print(classification_report(not_go_true, not_go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.4483    0.4062    0.4262        32
     class 1     0.5366    0.5789    0.5570        38

   micro avg     0.5000    0.5000    0.5000        70
   macro avg     0.4924    0.4926    0.4916        70
weighted avg     0.4962    0.5000    0.4972        70



## 2對2時 取 前三多者

In [333]:
not_go_true = []
not_go_predict = []


for j in range(len(predictlist[0])):
    tem_list = []
    
    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])
    if tem_list.count(0) ==2:
        if tem_list[0:3].count(0) ==2:
            not_go_true.append(y_true[j])
            not_go_predict.append(0)
        elif tem_list[0:3].count(1) ==2:
            not_go_true.append(y_true[j])
            not_go_predict.append(1)
print(classification_report(not_go_true, not_go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.5882    0.5263    0.5556        19
     class 1     0.4375    0.5000    0.4667        14

   micro avg     0.5152    0.5152    0.5152        33
   macro avg     0.5129    0.5132    0.5111        33
weighted avg     0.5243    0.5152    0.5178        33



# 試試看 不一樣的test train sample 來看看準確率

In [382]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 143,  min_samples_split = 4,  min_samples_leaf = 7,  max_depth = 90,  criterion =  'entropy' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099, solver  = 'liblinear')
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))


f1_weigted_list = [] #using to caluculate performance of the classfication task
precision_list = []
recall_list = []
go_percentage = []

for randomstate_seed in range(0,43):
    #split
    x_train,x_test,y_train,y_test = tt_split( X_no_ma ,y_no_ma,test_size=0.2,random_state=randomstate_seed)
    
    #fit and predict
    y_true = list(y_test)
    target_names = ['class 0', 'class 1']
    predictlist = []
    for i in range(len(clf_list)):
        clf = clf_list[i]
        clf.fit(x_train, y_train)
        y_pred = list( clf.predict(x_test) )
        predictlist.append(y_pred)
        
        
    # 4個一致
    go_true = []
    go_predict = []


    for j in range(len(predictlist[0])):
        tem_list = []

        for i in range(len(predictlist)):
            tem_list.append(predictlist[i][j])

        if tem_list.count(0) ==4 :
            go_true.append(y_true[j])
            go_predict.append(0)
        elif tem_list.count(1) ==4:
            go_true.append(y_true[j])
            go_predict.append(1)
    dict_output = (classification_report(go_true, go_predict, target_names=target_names, digits = 4, output_dict =True))
    f1_weigted =dict_output['weighted avg']['f1-score']
    precision = dict_output['weighted avg']['precision']
    recall = dict_output['weighted avg']['recall']
    
    print(f1_weigted, precision, recall, '出手率=', len(go_predict) / len(y_true) )
    f1_weigted_list.append(f1_weigted)
    precision_list.append(precision)
    recall_list.append(recall)
    go_percentage.append( len(go_predict) / len(y_true) )
np.mean(f1_weigted_list), np.mean(precision_list), np.mean(recall_list)

0.6966010733452594 0.7343061255509957 0.7209302325581395 出手率= 0.2945205479452055
0.6374683101669726 0.637037037037037 0.6388888888888888 出手率= 0.2465753424657534
0.584045584045584 0.5799373040752351 0.6 出手率= 0.273972602739726
0.5500751859235621 0.6707050645481628 0.5660377358490566 出手率= 0.363013698630137
0.531244535120956 0.5514550264550265 0.55 出手率= 0.410958904109589
0.5755128562146106 0.5725457338360564 0.5952380952380952 出手率= 0.2876712328767123
0.5043478260869565 0.531986531986532 0.5263157894736842 出手率= 0.2602739726027397
0.6461538461538462 0.6632860040567953 0.6521739130434783 出手率= 0.3150684931506849
0.6284343405494944 0.6233674272226594 0.6341463414634146 出手率= 0.2808219178082192
0.7288977159880834 0.7438423645320196 0.75 出手率= 0.2465753424657534
0.5524118738404453 0.594348508634223 0.5918367346938775 出手率= 0.3356164383561644
0.5332815332815334 0.5591675266597868 0.5277777777777778 出手率= 0.2465753424657534
0.5964193751140489 0.5990990990990991 0.5945945945945946 出手率= 0.253424657534246

(0.6078223160280873, 0.646472713632967, 0.625849944085211)

In [385]:
weights_list = []
sum_ = sum(go_percentage)

for i in range(43):
    weights_list.append(go_percentage[i]/ sum_)

In [387]:
np.average(f1_weigted_list, weights = weights_list), np.average(precision_list, weights = weights_list), np.average(recall_list, weights = weights_list)

(0.6054820393136178, 0.6443852685835642, 0.6242171189979124)

##  看看 有 ma的

In [395]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 143,  min_samples_split = 4,  min_samples_leaf = 7,  max_depth = 90,  criterion =  'entropy' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099, solver  = 'liblinear')
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))


f1_weigted_list = [] #using to caluculate performance of the classfication task
precision_list = []
recall_list = []
go_percentage = []


for randomstate_seed in range(0,43):
    #split
    x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=randomstate_seed)
    
    #fit and predict
    y_true = list(y_test)
    target_names = ['class 0', 'class 1']
    predictlist = []
    for i in range(len(clf_list)):
        clf = clf_list[i]
        clf.fit(x_train, y_train)
        y_pred = list( clf.predict(x_test) )
        predictlist.append(y_pred)
        
        
    # 4個一致
    go_true = []
    go_predict = []


    for j in range(len(predictlist[0])):
        tem_list = []

        for i in range(len(predictlist)):
            tem_list.append(predictlist[i][j])

        if tem_list.count(0) ==4 :
            go_true.append(y_true[j])
            go_predict.append(0)
        elif tem_list.count(1) ==4:
            go_true.append(y_true[j])
            go_predict.append(1)
    dict_output = (classification_report(go_true, go_predict, target_names=target_names, digits = 4, output_dict =True))
    f1_weigted =dict_output['weighted avg']['f1-score']
    precision = dict_output['weighted avg']['precision']
    recall = dict_output['weighted avg']['recall']
    
    print(f1_weigted, precision, recall, '出手率=', len(go_predict) / len(y_true) )
    f1_weigted_list.append(f1_weigted)
    precision_list.append(precision)
    recall_list.append(recall)
    go_percentage.append( len(go_predict) / len(y_true) )
np.mean(f1_weigted_list), np.mean(precision_list), np.mean(recall_list)

0.6696969696969697 0.6695075757575758 0.6818181818181818 出手率= 0.3013698630136986
0.6265734265734267 0.6363636363636364 0.6363636363636364 出手率= 0.22602739726027396
0.5591954022988506 0.5616071428571429 0.575 出手率= 0.273972602739726
0.5538461538461538 0.7011834319526626 0.5769230769230769 出手率= 0.3561643835616438
0.5405113291326124 0.6075363714163456 0.5614035087719298 出手率= 0.3904109589041096
0.5714285714285714 0.5714285714285714 0.5714285714285714 出手率= 0.2876712328767123
0.5276485788113695 0.5297157622739018 0.5348837209302325 出手率= 0.2945205479452055
0.6440376303076533 0.6592391304347825 0.6521739130434783 出手率= 0.3150684931506849
0.5995743387047734 0.5924978687127025 0.6086956521739131 出手率= 0.3150684931506849
0.6501606501606502 0.6651910531220876 0.6756756756756757 出手率= 0.2534246575342466
0.537878787878788 0.5747863247863249 0.5833333333333334 出手率= 0.3287671232876712
0.5973856209150327 0.6615384615384615 0.6 出手率= 0.23972602739726026
0.6112054329371817 0.6245024325519681 0.6052631578947368

(0.6015154036076626, 0.6410282854427541, 0.619823902131824)

In [396]:
weights_list = []
sum_ = sum(go_percentage)

for i in range(43):
    weights_list.append(go_percentage[i]/ sum_)
np.average(f1_weigted_list, weights = weights_list), np.average(precision_list, weights = weights_list), np.average(recall_list, weights = weights_list)

(0.6002183595635489, 0.6405666994108293, 0.6192973256423703)

# 試試看重新找有MA之下的最佳模型組合，給定隨機種子0

In [399]:
X.shape

(730, 950)

In [400]:
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=0)

### RF

In [407]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score 
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

# criterion to choose tree
criterions = ['gini', 'entropy'] 

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'criterion' : criterions,d
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'random_state' : [42]}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1000, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
rf_random.fit(x_train, y_train)
print(rf_random.best_params_)
print(rf_random.best_score_ )
clf =  rf_random.best_estimator_ 

y_pred = clf.predict(x_test)

y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 277 tasks      | elapsed:   21.7s
[Parallel(n_jobs=4)]: Done 588 tasks      | elapsed:   46.7s
[Parallel(n_jobs=4)]: Done 871 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 1236 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 1681 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done 2208 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done 2815 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 3504 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 4273 tasks      | elapsed:  5.6min
[Parallel(n_jobs=4)]: Done 5000 out of 5000 | elapsed:  6.5min finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0

{'random_state': 42, 'n_estimators': 30, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': 70, 'criterion': 'entropy', 'bootstrap': False}
0.5394326182512897
              precision    recall  f1-score   support

     class 0       0.52      0.38      0.44        68
     class 1       0.56      0.69      0.62        78

   micro avg       0.55      0.55      0.55       146
   macro avg       0.54      0.54      0.53       146
weighted avg       0.54      0.55      0.54       146



### svm

In [402]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score 
import numpy as np

kernel = ['linear', 'poly', 'rbf', 'sigmoid']
degree  = [1,2,3,4]
C = [float(x) for x in np.linspace(start = 0, stop = 1, num = 1000)]


# Create the random grid
random_grid = {'kernel': kernel,
               'degree' : degree,
               'C': C,
                }

clf = SVC()
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 200, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
clf_random.fit(x_train, y_train)

#print
print(clf_random.best_params_)
print(clf_random.best_score_ )
clf =  clf_random.best_estimator_ 
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   10.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   40.9s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  3.3min finished


{'kernel': 'linear', 'degree': 2, 'C': 0.7527527527527528}
0.5365325362195074
              precision    recall  f1-score   support

     class 0       0.55      0.50      0.52        68
     class 1       0.60      0.64      0.62        78

   micro avg       0.58      0.58      0.58       146
   macro avg       0.57      0.57      0.57       146
weighted avg       0.57      0.58      0.57       146



## logit

In [403]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
import numpy as np

penalty  = ['l1', 'l2']
C = [float(x) for x in np.linspace(start = 0, stop = 1, num = 1000)]


# Create the random grid
random_grid = {'penalty': penalty,
               'C': C,
                }

clf = LogisticRegression()
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 200, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
clf_random.fit(x_train, y_train)

#print
print(clf_random.best_params_)
print(clf_random.best_score_ )
clf =  clf_random.best_estimator_ 
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  39 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 459 tasks      | elapsed:    8.7s


{'penalty': 'l2', 'C': 0.3933933933933934}
0.5268773154393802
              precision    recall  f1-score   support

     class 0       0.51      0.44      0.47        68
     class 1       0.56      0.63      0.59        78

   micro avg       0.54      0.54      0.54       146
   macro avg       0.54      0.53      0.53       146
weighted avg       0.54      0.54      0.54       146



[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   19.1s finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### KNN

In [404]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
import numpy as np

n_neighbors  = [int(x) for x in np.linspace(start = 3, stop = 9, num = 4)]


p = [1,2,3] 

algorithm = [ 'ball_tree', 'kd_tree','brute']

leaf_size  = [int(x) for x in np.linspace(start = 2, stop = 60, num = 30)]


# Create the random grid
random_grid = {'n_neighbors': n_neighbors,
               'p' : p,
               'algorithm': algorithm,
               'leaf_size': leaf_size
              }

clf = KNeighborsClassifier()
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 200, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
clf_random.fit(x_train, y_train)

#print
print(clf_random.best_params_)
print(clf_random.best_score_ )
clf =  clf_random.best_estimator_ 
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   22.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  7.6min


{'p': 2, 'n_neighbors': 3, 'leaf_size': 48, 'algorithm': 'ball_tree'}
0.513182427063241
              precision    recall  f1-score   support

     class 0       0.49      0.57      0.53        68
     class 1       0.56      0.47      0.51        78

   micro avg       0.52      0.52      0.52       146
   macro avg       0.52      0.52      0.52       146
weighted avg       0.53      0.52      0.52       146



[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed: 10.5min finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [410]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 30,  min_samples_split = 4,  min_samples_leaf = 1,  max_depth = 70,  criterion =  'entropy' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.3933933933933934, solver  = 'liblinear')
svm = SVC(kernel = 'linear', degree =2, C = 0.7527527527527528)
clf_list += [rf,logistic, svm ]
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))


#fit and predict
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for i in range(len(clf_list)):
    clf = clf_list[i]
    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)


# 4個一致
go_true = []
go_predict = []


for j in range(len(predictlist[0])):
    tem_list = []

    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])

    if tem_list.count(0) ==4 :
        go_true.append(y_true[j])
        go_predict.append(0)
    elif tem_list.count(1) ==4:
        go_true.append(y_true[j])
        go_predict.append(1)
print(classification_report(go_true, go_predict, target_names=target_names, digits = 4))

              precision    recall  f1-score   support

     class 0     0.4667    0.4118    0.4375        17
     class 1     0.6429    0.6923    0.6667        26

   micro avg     0.5814    0.5814    0.5814        43
   macro avg     0.5548    0.5520    0.5521        43
weighted avg     0.5732    0.5814    0.5761        43



## without flat and ma

In [417]:
features_dict = {}
for date in market_date_list:
    features_dict[date] = [] 
    
    
feature_list = []
for item in ['news', 'bbs', 'forum']:    
    if item == 'bbs':
        article = bbs_content_plus_title
        article_number = bbs_number_of_document
    elif item == 'forum':
        article = forum_content_plus_title
        article_number = forum_number_of_document
    else:
        article = news_content_plus_title
        article_number = news_number_of_document
        
    for a_class in ['up', 'down']:
        keyword_list = df[item+ '_' + a_class].dropna().tolist()
        
        
        feature_list += keyword_list
                
        for date in market_date_list:
            for keyword in keyword_list:
                
                try:
                    feature_i = article[date][a_class].count(keyword) / article_number[date][a_class]
                except:
                    feature_i = 0
                features_dict[date].append(    feature_i )
                    
                    
df_ready = pd.DataFrame.from_dict(features_dict, orient = 'index', columns  = feature_list)
dropped_columns_index = []
for i in range(len(df_ready.columns)):
    if df_ready.iloc[:,i].tolist().count(0) == 732:
        dropped_columns_index.append(i)
kept_colums = list(range(len(df_ready.columns)))
for i in dropped_columns_index:
    kept_colums.remove(i)
df_ready4 = df_ready.iloc[:,kept_colums]
y = pd.DataFrame.from_dict(target, orient = 'index', columns  = ['target'])

In [418]:
new_features = pd.DataFrame.from_dict(new_features_dict, orient = 'index', columns  = ['ma\%', 'news\%','bbs\%','forum\%'])
for i in [ 'news\%','bbs\%','forum\%']:
    df_ready4[i] = new_features[i]

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [421]:
df_ready4['target'] = y

from sklearn.model_selection import train_test_split as tt_split

X_no_flat = df_ready4.iloc[2:,0:-1]
y_no_flat = df_ready4.iloc[2:,:]['target']
x_train,x_test,y_train,y_test = tt_split(X_no_flat,y_no_flat,test_size=0.2,random_state=0)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [423]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 143,  min_samples_split = 4,  min_samples_leaf = 7,  max_depth = 90,  criterion =  'entropy' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099, solver  = 'liblinear')
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))






#fit and predict
y_true = list(y_test)
target_names = ['class 0', 'class 1']
predictlist = []
for i in range(len(clf_list)):
    clf = clf_list[i]
    clf.fit(x_train, y_train)
    y_pred = list( clf.predict(x_test) )
    predictlist.append(y_pred)


# 4個一致
go_true = []
go_predict = []


for j in range(len(predictlist[0])):
    tem_list = []

    for i in range(len(predictlist)):
        tem_list.append(predictlist[i][j])

    if tem_list.count(0) ==4 :
        go_true.append(y_true[j])
        go_predict.append(0)
    elif tem_list.count(1) ==4:
        go_true.append(y_true[j])
        go_predict.append(1)
print(classification_report(go_true, go_predict, target_names=target_names, digits = 4))


              precision    recall  f1-score   support

     class 0     0.5556    0.2778    0.3704        18
     class 1     0.6061    0.8333    0.7018        24

   micro avg     0.5952    0.5952    0.5952        42
   macro avg     0.5808    0.5556    0.5361        42
weighted avg     0.5844    0.5952    0.5597        42



## rf for without ma new!!


In [439]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score 
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

# criterion to choose tree
criterions = ['gini', 'entropy'] 

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'criterion' : criterions,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'random_state' : [42]}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1000, 
                               cv = 5, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
rf_random.fit(x_train, y_train)
print(rf_random.best_params_)
print(rf_random.best_score_ )
clf =  rf_random.best_estimator_ 

y_pred = clf.predict(x_test)

y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   12.6s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   22.0s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   37.7s
[Parallel(n_jobs=4)]: Done 820 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 1550 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done 2440 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 3494 tasks      | elapsed:  4.4min
[Parallel(n_jobs=4)]: Done 4449 tasks      | elapsed:  5.5min


{'random_state': 42, 'n_estimators': 10, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_depth': 70, 'criterion': 'gini', 'bootstrap': False}
0.5610104248784865
              precision    recall  f1-score   support

     class 0       0.56      0.41      0.47        68
     class 1       0.58      0.72      0.64        78

   micro avg       0.58      0.58      0.58       146
   macro avg       0.57      0.56      0.56       146
weighted avg       0.57      0.58      0.56       146



[Parallel(n_jobs=4)]: Done 5000 out of 5000 | elapsed:  6.1min finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [441]:
clf_list = []
rf = RandomForestClassifier(n_estimators = 10,  min_samples_split = 6,  min_samples_leaf = 4,  max_depth = 70,  criterion =  'gini' ,  bootstrap = False, random_state = 42)
logistic = LogisticRegression(penalty = 'l2', C = 0.9099099099099099, solver  = 'liblinear')
svm = SVC(kernel = 'linear', degree =2, C = 0.7187187187187187)
clf_list += [rf,logistic, svm ]
clf_list.append(KNeighborsClassifier(p = 2,  n_neighbors = 3,  leaf_size = 48,  algorithm =  'ball_tree'  ))


0.6590122106797887 0.6574500768049154 0.6666666666666666 出手率= 0.3082191780821918
0.6158916890624209 0.6172064777327936 0.6153846153846154 出手率= 0.2671232876712329
0.6216216216216215 0.6216216216216216 0.6216216216216216 出手率= 0.2534246575342466
0.5268638013667661 0.5478139793929268 0.5370370370370371 出手率= 0.3698630136986301
0.5880970625798212 0.6105499438832772 0.5925925925925926 出手率= 0.3698630136986301
0.48879787514446177 0.49179536679536673 0.4864864864864865 出手率= 0.2534246575342466
0.5477428180574555 0.5452220413857243 0.5588235294117647 出手率= 0.2328767123287671
0.6816326530612244 0.7142857142857143 0.6904761904761905 出手率= 0.2876712328767123
0.5373040752351097 0.5266666666666666 0.55 出手率= 0.273972602739726
0.7314285714285713 0.7542857142857142 0.7428571428571429 出手率= 0.23972602739726026
0.6727859935407107 0.7054054054054053 0.7027027027027027 出手率= 0.2534246575342466
0.5428571428571428 0.5428571428571428 0.5428571428571428 出手率= 0.23972602739726026
0.6144691881701426 0.6306620209059234 0

(0.619189403104275, 0.6394728653925609, 0.6261980378042465)